In [2]:
import pandas as pd
import numpy as np
import re
import requests
import bs4
import lxml
import plotly.express as px
import time
from playwright.sync_api import sync_playwright
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException, TimeoutException

pd.options.plotting.backend = 'plotly'

In [3]:
driver = webdriver.Chrome()


In [4]:
url = r'https://www.nba.com/stats/players/advanced?SeasonType=Regular+Season'

In [5]:
driver.get(url)

In [6]:
select = Select(driver.find_element('xpath',r"/html/body/div[1]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select"))

In [7]:
select.select_by_index(0)

In [8]:
src = driver.page_source
parser = bs4.BeautifulSoup(src, 'html.parser')
table = parser.find('table', attrs={'class': 'Crom_table__p1iZz'})
tt = table.find('tbody', attrs={'class':'Crom_body__UYOcU'})
players = tt.find_all('tr')
playerList = [p.find_all('td')[1] for p in players]
playerList = [p.find('a') for p in players]
names = [a.text for a in playerList]
links = [a.get('href') for a in playerList]



In [9]:
df = pd.DataFrame(index=names, data = {'url':links})

In [10]:
driver = webdriver.Chrome()


statsDict = {
    'NAME' : [],
    'GAME' : [],
    'W/L' : [],
    'MINUTES' : [],
    'OFFRTG' : [],
    'DEFRTG' : [],
    'NETRTG' : [],
    'AST%' : [],
    'AST/TO' : [],
    'AST RATIO' : [],
    'OREB%' : [],
    'DREB%' : [],
    'REB%' : [],
    'TO RATIO' : [],
    'EFG%' : [],
    'TS%' : [],
    'USG%' : [],
    'PACE' : [],
    'PIE' : [], 
}

for i in range(0,df.shape[0]):
    check = 0
    playerURL = f'https://www.nba.com{df["url"].iloc[i]}/boxscores-advanced'
    driver.get(playerURL)
    
    try:
        alert = driver.switch_to.alert
        alert.dismiss()
    except:
        pass
    
    for attempt in range(3):
        try:
            WebDriverWait(driver, 8).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[2]/label/div/select")))
            break
        except Exception as e:
            driver.refresh() 
            time.sleep(1)  
            
    seaSelect = Select(driver.find_element('xpath',r"/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[2]/label/div/select"))
    seaSelect.select_by_value('Regular Season')
    time.sleep(1)
    regSelect = Select(driver.find_element('xpath',r"/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[2]/label/div/select"))
    regSelect.select_by_value('Regular Season')
    time.sleep(1)
    driver.refresh()
    time.sleep(1)

    for attempt in range(3):
        try:
            WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, r"/html/body/div[1]/div[2]/div[2]/section/div[4]/section[3]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select")))
            check = 1
            break  
        except (NoSuchElementException, TimeoutException) as e:
            driver.refresh() 
            time.sleep(1)  

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Crom_body__UYOcU")))        
    except:
            try:
                driver.refresh()
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Crom_body__UYOcU")))
            except:
                try:
                    driver.refresh()
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Crom_body__UYOcU")))
                except:
                    continue
            
    if check == 1:
        allSelect = driver.find_element('xpath',r"/html/body/div[1]/div[2]/div[2]/section/div[4]/section[3]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select")
        if ('-1' in [option.get_attribute("value") for option in Select(allSelect).options]) and (allSelect is not None):
            Select(allSelect).select_by_value('-1')
        
    src = driver.page_source
    table = bs4.BeautifulSoup(src, 'html.parser')
    statTable = table.find('tbody', attrs={'class': 'Crom_body__UYOcU'})
    games = statTable.find_all('tr')
    
    for game in games:
        stats = game.find_all('td')
        stats = [x.text for x in stats]
        statsDict['NAME'].append(df.index[i])
        statsDict['GAME'].append(stats[0])
        statsDict['W/L'].append(stats[1])
        statsDict['MINUTES'].append(float(stats[2]))
        statsDict['OFFRTG'].append(float(stats[3]))
        statsDict['DEFRTG'].append(float(stats[4]))
        statsDict['NETRTG'].append(float(stats[5]))
        statsDict['AST%'].append(float(stats[6]))
        statsDict['AST/TO'].append(float(stats[7]))
        statsDict['AST RATIO'].append(float(stats[8]))
        statsDict['OREB%'].append(float(stats[9]))
        statsDict['DREB%'].append(float(stats[10]))
        statsDict['REB%'].append(float(stats[11]))
        statsDict['TO RATIO'].append(float(stats[12]))
        statsDict['EFG%'].append(float(stats[13]))
        statsDict['TS%'].append(float(stats[14]))
        statsDict['USG%'].append(float(stats[15]))
        statsDict['PACE'].append(float(stats[16]))
        statsDict['PIE'].append(float(stats[17]))

driver.close()
statsDict

KeyboardInterrupt: 

In [69]:
statDF = pd.DataFrame(data = statsDict)
statDF
statDF.to_csv('advanced.csv', index=False)

In [70]:
statDF

,NAME,GAME,W/L,MINUTES,OFFRTG,DEFRTG,NETRTG,AST%,AST/TO,AST RATIO,OREB%,DREB%,REB%,TO RATIO,EFG%,TS%,USG%,PACE,PIE
0,Buddy Hield,"Apr 14, 2024 - PHI vs. BKN",W,22.0,118.2,76.2,42.0,33.3,5.0,22.7,7.7,19.2,13.5,4.5,53.3,58.2,32.7,94.90,22.2
1,Buddy Hield,"Apr 12, 2024 - PHI vs. ORL",W,14.0,112.9,109.4,3.5,0.0,0.0,0.0,5.6,15.4,9.7,16.7,60.0,60.0,17.1,106.11,5.2
2,Buddy Hield,"Apr 09, 2024 - PHI vs. DET",W,26.0,132.7,92.9,39.9,0.0,0.0,0.0,6.5,14.8,10.3,7.1,70.8,69.9,20.9,104.68,11.0
3,Buddy Hield,"Apr 07, 2024 - PHI @ SAS",W,20.0,100.0,102.5,-2.5,23.1,1.5,23.1,4.2,4.5,4.3,15.4,57.1,63.5,21.7,99.15,12.0
4,Buddy Hield,"Apr 06, 2024 - PHI @ MEM",W,26.0,103.7,78.9,24.8,5.6,1.0,7.1,0.0,3.0,1.7,7.1,29.2,29.2,21.7,104.21,-1.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26396,Javonte Smart,"Dec 22, 2023 - PHI vs. TOR",W,1.0,0.0,150.0,-150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,108.00,0.0
26397,Kaiser Gates,"Oct 30, 2023 - NOP vs. GSW",L,7.0,81.3,88.2,-7.0,0.0,0.0,0.0,0.0,9.1,4.5,0.0,0.0,0.0,22.2,106.55,-15.0
26398,Malcolm Cazalon,"Nov 05, 2023 - DET vs. PHX",L,3.0,183.3,100.0,83.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,111.48,0.0
26399,Markquis Nowell,"Nov 15, 2023 - TOR vs. MIL",L,4.0,128.6,62.5,66.1,100.0,0.0,50.0,50.0,0.0,28.6,0.0,0.0,53.2,22.2,102.37,35.7


In [73]:
driver = webdriver.Chrome()

tradDict = {
    'NAME' : [],
    'GAME' : [],
    'W/L' : [],
    'MIN' : [],
    'PTS' : [],
    'FGM' : [],
    'FGA' : [],
    'FG%' : [],
    '3PM' : [],
    '3PA' : [],
    '3P%' : [],
    'FTM' : [],
    'FTA' : [],
    'FT%' : [],
    'OREB' : [],
    'DREB' : [],
    'REB' : [],
    'AST' : [],
    'STL' : [],
    'BLK' : [],
    'TOV' : [],
    'PF' : [],
    '+/-' : [],
}

for i in range(0,df.shape[0]):
    check = 0
    playerURL = f'https://www.nba.com{df["url"].iloc[i]}/boxscores-traditional'
    driver.get(playerURL)
    
    try:
        alert = driver.switch_to.alert
        alert.dismiss()
    except:
        pass
    
    for attempt in range(3):
        try:
            WebDriverWait(driver, 8).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[2]/label/div/select")))
            break
        except Exception as e:
            driver.refresh() 
            time.sleep(1)  
            
    regSelect = Select(driver.find_element('xpath',r"/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[2]/label/div/select"))
    regSelect.select_by_value('Regular Season')
    time.sleep(1)
    driver.refresh()
    time.sleep(1)

    for attempt in range(3):
        try:
            WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, r"/html/body/div[1]/div[2]/div[2]/section/div[4]/section[3]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select")))
            check = 1
            break  
        except (NoSuchElementException, TimeoutException) as e:
            driver.refresh() 
            time.sleep(1)  

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Crom_body__UYOcU")))        
    except:
            try:
                driver.refresh()
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Crom_body__UYOcU")))
            except:
                try:
                    driver.refresh()
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Crom_body__UYOcU")))
                except:
                    continue
            
    if check == 1:
        allSelect = driver.find_element('xpath',r"/html/body/div[1]/div[2]/div[2]/section/div[4]/section[3]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select")
        if ('-1' in [option.get_attribute("value") for option in Select(allSelect).options]) and (allSelect is not None):
            Select(allSelect).select_by_value('-1')
        
    src = driver.page_source
    table = bs4.BeautifulSoup(src, 'html.parser')
    statTable = table.find('tbody', attrs={'class': 'Crom_body__UYOcU'})
    games = statTable.find_all('tr')
    
    
    for game in games:
        stats = game.find_all('td')
        stats = [x.text for x in stats]
        tradDict['NAME'].append(df.index[i])
        tradDict['GAME'].append(stats[0])
        tradDict['W/L'].append(stats[1])
        tradDict['MIN'].append((stats[2]))
        tradDict['PTS'].append(float(stats[3]))
        tradDict['FGM'].append(float(stats[4]))
        tradDict['FGA'].append(float(stats[5]))
        tradDict['FG%'].append(float(stats[6]))
        tradDict['3PM'].append(float(stats[7]))
        tradDict['3PA'].append(float(stats[8]))
        tradDict['3P%'].append(float(stats[9]))
        tradDict['FTM'].append(float(stats[10]))
        tradDict['FTA'].append(float(stats[11]))
        tradDict['FT%'].append(float(stats[12]))
        tradDict['OREB'].append(float(stats[13]))
        tradDict['DREB'].append(float(stats[14]))
        tradDict['REB'].append(float(stats[15]))
        tradDict['AST'].append(float(stats[16]))
        tradDict['STL'].append(float(stats[17]))
        tradDict['BLK'].append(float(stats[18]))
        tradDict['TOV'].append(float(stats[19]))
        tradDict['PF'].append(float(stats[20]))
        tradDict['+/-'].append(float(stats[21]))
        
driver.close()
tradDict

{'NAME': ['Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Buddy Hield',
  'Bud

In [75]:
len(list(set(tradDict['NAME'])))
len(tradDict['NAME'])/572

46.15559440559441

In [76]:
tradDF = pd.DataFrame(data = tradDict)
#statDF.drop(['NAME', 'GAME', 'W/L'], axis=1).mean()
tradDF
tradDF.to_csv('traditional.csv', index=False)

In [77]:
tradDF


,NAME,GAME,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
0,Buddy Hield,"Apr 14, 2024 - PHI vs. BKN",W,21:45,19.0,6.0,15.0,40.0,4.0,10.0,...,100.0,2.0,5.0,7.0,5.0,1.0,0.0,1.0,2.0,20.0
1,Buddy Hield,"Apr 12, 2024 - PHI vs. ORL",W,14:15,6.0,2.0,5.0,40.0,2.0,4.0,...,0.0,1.0,2.0,3.0,0.0,0.0,0.0,1.0,1.0,0.0
2,Buddy Hield,"Apr 09, 2024 - PHI vs. DET",W,25:27,18.0,6.0,12.0,50.0,5.0,9.0,...,50.0,2.0,4.0,6.0,0.0,0.0,0.0,1.0,2.0,21.0
3,Buddy Hield,"Apr 07, 2024 - PHI @ SAS",W,19:36,10.0,3.0,7.0,42.9,2.0,5.0,...,100.0,1.0,1.0,2.0,3.0,1.0,0.0,2.0,1.0,0.0
4,Buddy Hield,"Apr 06, 2024 - PHI @ MEM",W,25:34,7.0,3.0,12.0,25.0,1.0,7.0,...,0.0,0.0,1.0,1.0,1.0,2.0,1.0,1.0,3.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26396,Javonte Smart,"Dec 22, 2023 - PHI vs. TOR",W,0:40,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0
26397,Kaiser Gates,"Oct 30, 2023 - NOP vs. GSW",L,7:26,0.0,0.0,4.0,0.0,0.0,2.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,-2.0
26398,Malcolm Cazalon,"Nov 05, 2023 - DET vs. PHX",L,2:35,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
26399,Markquis Nowell,"Nov 15, 2023 - TOR vs. MIL",L,3:31,2.0,0.0,1.0,0.0,0.0,0.0,...,100.0,2.0,0.0,2.0,2.0,1.0,0.0,0.0,0.0,4.0
